# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features_5h.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(9058, 11841)


15h_dft_dom_cycle_ddt_lag39    268
15h_dft_dom_cycle_ddt_lag38    265
15h_dft_dom_cycle_dt_lag39     265
15h_dft_dom_cycle_lag39        262
15h_dft_dom_cycle_ddt_lag37    262
                              ... 
5h_comb_spectrum_pwr_37          0
5h_comb_spectrum_pwr_36          0
5h_comb_spectrum_pwr_35          0
5h_comb_spectrum_pwr_34          0
5h_comb_spectrum_pwr_26          0
Length: 11841, dtype: int64

In [2]:
import numpy as np
from jesse import utils

label = np.load("data/label_side.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label["side_label"] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-03 10:46:00,7303.97,7528.76,7580.08,7229.0,357578.646,1.0


In [3]:
side_features = df_features.iloc[268:]
side_label = df_label.iloc[268:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

(8790, 11841)
(8790,)


/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_45475/2938923821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


15h_ac_0                     0
100m_ac_4                    0
5h_williams_r_ddt_lag35      0
5h_williams_r_ddt_lag36      0
5h_williams_r_ddt_lag37      0
                            ..
5h_ac_2                      0
5h_ac_3                      0
5h_ac_4                      0
5h_ac_5                      0
100m_williams_r_ddt_lag39    0
Length: 11841, dtype: int64

In [4]:
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector(max_features=2500)
selector.fit(side_features, side_label)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: 15h_williams_r (最大F值: 3304.2423)
➤ 计算特征冗余度...
➤ 总计选择2500个特征 (已选择1个，还需选择2499个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2499/2499 [05:42<00:00,  7.29特征/s]



✅ 特征选择完成：从11841个特征中选择了2500个，舍弃了9341个
✅ 选择的特征: ['15h_williams_r', '100m_phase_accumulation_lag16', '5h_stc_dt', '15h_comb_spectrum_pwr_8', '5h_trendflex', '15h_bandpass_lag10', '5h_williams_r', '15h_trendflex', '15h_williams_r_lag1', '5h_hurst_coef_slow', '5h_williams_r_lag1', '5h_fisher', '15h_evenbetter_sinewave_long', '5h_acc_swing_index_dt', '5h_evenbetter_sinewave_long', '5h_trendflex_lag1', '15h_trendflex_lag1', '15h_comb_spectrum_pwr_10', '15h_fisher', '5h_williams_r_lag2', '15h_dual_diff_lag12', '15h_evenbetter_sinewave_short', '5h_fisher_lag1', '5h_evenbetter_sinewave_long_lag1', '100m_williams_r', '5h_comb_spectrum_dom_cycle_lag18', '5h_trendflex_lag2', '15h_trendflex_lag2', '5h_evenbetter_sinewave_short', '5h_hurst_coef_slow_lag38', '15h_stc', '100m_trendflex', '15h_williams_r_lag2', '15h_comb_spectrum_pwr_11', '5h_vwap_dt', '5h_evenbetter_sinewave_long_lag2', '15h_acp_pwr_1', '15h_evenbetter_sinewave_short_lag1', '100m_trendflex_lag1', '5h_trendflex_lag3', '5h_natr_dt', '5h

In [5]:
side_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
side_res

15h_williams_r                   3.304242e+03
5h_williams_r                    2.377262e+03
5h_trendflex                     2.342253e+03
15h_williams_r_lag1              2.074026e+03
5h_trendflex_lag1                1.975085e+03
                                     ...     
100m_fisher_ddt_lag38            4.636818e-07
100m_mod_stochastic_lag14        2.758933e-07
100m_hurst_coef_fast_dt_lag23    8.288909e-08
100m_williams_r_ddt_lag8         7.734311e-08
15h_dual_diff_lag32              1.825017e-08
Length: 11841, dtype: float64

In [6]:
import json

with open(f"data/side_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(side_res.to_dict(), f, indent=4)

In [7]:
import json

import pandas as pd

with open(f"data/side_features_FCQSelector.json", "r") as f:
    side_res = pd.Series(json.load(f))

In [8]:
DOLLAR_BAR_SHORT_TERM = "100m"
DOLLAR_BAR_MEDIUM_TERM = "5h"
DOLLAR_BAR_LONG_TERM = "15h"

feature_info = {
    "side": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
    },
    "meta": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
        "model_res": [],
    },
}

for k, v in side_res[:2500].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["side"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["side"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["side"][DOLLAR_BAR_LONG_TERM].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("model/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [9]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_5h.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = lgb.Booster(model_file="model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob

assert df_features.shape[0] == meta_label.shape[0]
assert "side_model_res" in df_features.columns

df_features.isna().sum(axis=0).sort_values(ascending=False)

15h_dft_dom_cycle_ddt_lag39    268
15h_dft_dom_cycle_ddt_lag38    265
15h_dft_dom_cycle_dt_lag39     265
15h_dft_dom_cycle_lag39        262
15h_dft_dom_cycle_ddt_lag37    262
                              ... 
5h_comb_spectrum_pwr_14          0
5h_comb_spectrum_pwr_13          0
5h_comb_spectrum_pwr_12          0
5h_comb_spectrum_pwr_11          0
side_model_res                   0
Length: 11842, dtype: int64

In [10]:
meta_features = df_features[268:]
meta_label = meta_label[268:]

meta_label = pd.Series(meta_label[:, 6].astype(int), index=meta_features.index)

# meta_features = meta_features[meta_label["ret"].notna()]
# meta_label = meta_label[meta_label["ret"].notna()]["bin"]

print(meta_features.shape)
print(meta_label.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(8790, 11842)
(8790,)


15h_ac_0                   0
100m_ac_4                  0
5h_williams_r_ddt_lag35    0
5h_williams_r_ddt_lag36    0
5h_williams_r_ddt_lag37    0
                          ..
5h_ac_2                    0
5h_ac_3                    0
5h_ac_4                    0
5h_ac_5                    0
side_model_res             0
Length: 11842, dtype: int64

In [11]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector(max_features=2500)
selector.fit(meta_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: 15h_vwap_lag28 (最大F值: 578.6306)
➤ 计算特征冗余度...
➤ 总计选择2500个特征 (已选择1个，还需选择2499个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2499/2499 [05:42<00:00,  7.30特征/s]



✅ 特征选择完成：从11842个特征中选择了2500个，舍弃了9342个
✅ 选择的特征: ['15h_vwap_lag28', '15h_homodyne_dt_lag31', '5h_evenbetter_sinewave_long_ddt_lag14', '15h_dft_dom_cycle_ddt_lag34', '15h_adaptive_stochastic_dt_lag37', '5h_adaptive_bp_ddt_lag32', '5h_acc_swing_index_lag1', '15h_hurst_coef_fast_dt_lag3', '15h_vwap_lag37', '5h_acp_pwr_38', '15h_vwap_lag30', '5h_homodyne_ddt_lag37', '5h_acc_swing_index_lag2', '5h_evenbetter_sinewave_short_ddt', '15h_vwap_lag27', '5h_hurst_coef_slow_ddt_lag31', '15h_vwap_lag29', '5h_comb_spectrum_dom_cycle_ddt_lag12', '15h_vwap_lag31', '5h_dft_dom_cycle_dt_lag7', '5h_acc_swing_index_lag3', '15h_vwap_lag38', '15h_vwap_lag26', '5h_acp_pwr_9', '5h_acc_swing_index_lag4', '15h_vwap_lag36', '100m_evenbetter_sinewave_long_dt_lag21', '15h_adaptive_stochastic_ddt_lag18', '15h_vwap_lag32', '5h_acc_swing_index_lag5', '5h_hurst_coef_fast_dt_lag16', '15h_vwap_lag35', '5h_vwap_ddt_lag28', '15h_vwap_lag34', '5h_acc_swing_index', '15h_vwap_lag33', '15h_vwap_lag39', '100m_ehlers_early_onset_t

In [14]:
import json

with open(f"data/meta_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(meta_res.to_dict(), f, indent=4)

In [15]:
import json

import pandas as pd

with open("data/meta_features_FCQSelector.json", "r") as f:
    meta_res = pd.Series(json.load(f))

In [16]:
meta_res.to_dict()["side_model_res"]

9.614614618143191

In [18]:
from pathlib import Path

feature_info = Path("model/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

DOLLAR_BAR_SHORT_TERM = "100m"
DOLLAR_BAR_MEDIUM_TERM = "5h"
DOLLAR_BAR_LONG_TERM = "15h"


feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = []
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = []
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = []
feature_info["meta"]["model_res"] = []

for k, v in meta_res[:2500].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        feature_info["meta"]["model_res"].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("model/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)